Установка библиотек для работы с файлами Excel

In [ ]:
!pip install xlrd openpyxl

# Практика № 2
### Третья пара по обработке данных (24.10.2022)

Файлы, с которыми работаем в данном ноутбуке в телеграм-канале.

Конечная цель: расчитать промежуточную успеваемость студентов 1 курса по математике

Цели на пару:
* научиться работать с таблицами Excel,
* познакомиться с мультииндексами,
* научиться соединять несколько таблиц.

In [1]:
import pandas as pd

Читаем файл с успеваемостью

В методе есть атрибут <code>sheet_name</code>, который отвечает за порядковый номер или название листа, который необходимо прочитать.

По умолчанию открывается первый лист.

In [2]:
students = pd.read_excel('student_marks.xlsx', header=None)

Далее выбираем строки, которые будем использовать в качестве закоголов / названий колонок.

В исходном файле заголовок в 3 строки, можно использовать 3х-уровневый заголовок, можно сократиться до двух уровней.

<code> df.T </code> - транспонирует таблицу: строки становятся столбцами, а столбцы - строками. Сделано просто для удобства.

Метод `df.ffill()` - расшифровывается как forward fill - заполнение столбца протягиванием имеющихся значений вперед, вместо `nan`\`ов

In [3]:
columns = students.loc[[1, 2]].T
columns = columns.ffill(limit=1)

Преобразовываем подготовленный сет в мультииндекс и устанавливаем его в название колонок

In [4]:
students.columns = pd.MultiIndex.from_frame(columns)

Удаляем из сета "заголовочные" строки

In [5]:
students = students.drop(index=[0, 1, 2])

Уставливаем ID студента индексом и удаляем эту колонку из сета.

`df[column_name].squeeze()` - преобразует колонку-сет в pd.Series 

In [6]:
students = students.set_index(students[('Тема пары','ID')].squeeze())
students = students.drop(columns=('Тема пары','ID'))

/tmp/ipykernel_33406/1804181330.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  students = students.set_index(students[('Тема пары','ID')].squeeze())


In [8]:
students.head()

1               Вступ-е тестирование        \
2                            посещ-е актив   
(Тема пары, ID)                              
i22s0633                           1   NaN   
i22s0105                           1   NaN   
i22s0897                           1   NaN   
i22s0889                           0   NaN   
i22s0623                           1   NaN   

1               Степени с натур. и целым показателем        \
2                                            посещ-е актив   
(Тема пары, ID)                                              
i22s0633                                           1   NaN   
i22s0105                                           1     1   
i22s0897                                           1   NaN   
i22s0889                                         0.5   NaN   
i22s0623                                           1   NaN   

1               Степени с рацион-м показателем       Многочлены        \
2                                      посещ-е актив    посещ-е актив   
(Тема пары, ID)                                                         
i22s0633                                     1     1          1     1   
i22s0105                                     1     1          1     1   
i22s0897                                     1   NaN          1   NaN   
i22s0889                                     1   NaN          1   NaN   
i22s0623                                     1   NaN          1   NaN   

1               Квадратный трехчлен. Т-ма Виета        ...     NaN        \
2                                       посещ-е актив  ... посещ-е актив   
(Тема пары, ID)                                        ...                 
i22s0633                                      1   NaN  ...     NaN   NaN   
i22s0105                                      1     1  ...     NaN   NaN   
i22s0897                                      1   NaN  ...     NaN   NaN   
i22s0889                                      1     1  ...     NaN   NaN   
i22s0623                                      1   NaN  ...     NaN   NaN   

1                                                                        
2               посещ-е актив посещ-е актив посещ-е актив посещ-е актив  
(Тема пары, ID)                                                          
i22s0633            NaN   NaN     NaN   NaN     NaN   NaN     NaN   NaN  
i22s0105            NaN   NaN     NaN   NaN     NaN   NaN     NaN   NaN  
i22s0897            NaN   NaN     NaN   NaN     NaN   NaN     NaN   NaN  
i22s0889            NaN   NaN     NaN   NaN     NaN   NaN     NaN   NaN  
i22s0623            NaN   NaN     NaN   NaN     NaN   NaN     NaN   NaN  

[5 rows x 54 columns]

Меняем имя индекса

In [9]:
students.index.name = 'ID'

Удаляем все пустые колонки

In [10]:
students = students.dropna(how='all', axis=1)

Преобразуем типы

In [11]:
students = students.astype('float')

`students.columns.get_level_values(2)` - позволяет получить значение определенного уровня заголовка

Обращаемся либо по порядковому номеру, либо по лэйблу, здесь лэйблы целочисленные, поэтому при значениях 0 и 1 метод вернет первый уровень заголовка, тему пары, при значении 2 деление на посещение и активность.

Выбираем **только** колонки посещения и считаем сумму баллов по каждому студенту (строке). Суммируем столбцы, поэтому `axis=1`.

In [12]:
result = students.iloc[:, students.columns.get_level_values(2) == 'посещ-е'].sum(axis=1)

___
Читаем следующий файл с результатами домашней работы / теста.

In [13]:
test_marks = pd.read_excel('hw_1.xlsx', index_col='ID') 

In [14]:
test_marks['Процент правильных ответов (%)']

ID
NaN           5.56
i22s0991     72.22
i22s0986     44.44
i22s0628     72.22
i22s0979     44.44
             ...  
i22s0903     38.89
i22s0627     77.78
i22s0991     22.22
i22s0993    100.00
NaN          33.33
Name: Процент правильных ответов (%), Length: 95, dtype: float64

Соединять таблицы можно разными методами:
* `concat`
* `merge`
* `join`

При `concat`\`е таблицы соединяются как бы по диагонали, и индексы дублируются, поэтому нужно удалить дубликаты и совместить значения строк из разных таблиц

In [15]:
df = pd.concat([result, test_marks], axis=0)
df.iloc[df.index.duplicated(), :]

,0,#,№,Дата завершения,Потрачено времени,Группа,Количество правильных ответов,Процент правильных ответов (%)
ID,,,,,,,,
i22s0633,NaN,25.0,151392349.0,2022-09-13 19:46:36.395,00:49:56,ИТ22.13,10.0,55.56
i22s0105,NaN,53.0,151170371.0,2022-09-12 10:32:56.595,01:34:36,ИТ22.13,0.0,0.00
i22s0897,NaN,3.0,151551656.0,2022-09-14 19:57:17.435,00:33:35,ИТ22.13,13.0,72.22
i22s0889,NaN,14.0,151423418.0,2022-09-13 23:42:53.409,00:19:01,ИТ22.13,8.0,44.44
i22s0623,NaN,37.0,151340060.0,2022-09-13 14:12:00.532,00:06:55,ИТ22.13,10.0,55.56
i22s0661,NaN,79.0,151070148.0,2022-09-10 15:54:38.375,00:07:03,ИТ22.13,7.0,38.89
i22s0893,NaN,67.0,151116986.0,2022-09-11 16:30:42.759,00:49:21,ИТ22.13,12.0,66.67
i22s0629,NaN,48.0,151243250.0,2022-09-12 18:41:50.139,00:22:29,ИТ22.13,15.0,83.33
i22s0906,NaN,74.0,151108459.0,2022-09-11 14:12:42.832,00:57:42,ИТ22.13,15.0,83.33


In [16]:
df.groupby(by=df.index).last().loc['i22s0633']

0                                                       15.0
#                                                       25.0
№                                                151392349.0
Дата завершения                   2022-09-13 19:46:36.395000
Потрачено времени                                   00:49:56
Группа                                               ИТ22.13
Количество правильных ответов                           10.0
Процент правильных ответов (%)                         55.56
Name: i22s0633, dtype: object

Соединение таблиц с помощью `join`

In [17]:
result.name = 'pos'
test_marks.join(result)

,#,№,Дата завершения,Потрачено времени,Группа,Количество правильных ответов,Процент правильных ответов (%),pos
ID,,,,,,,,
i22s0105,53,151170371,2022-09-12 10:32:56.595,01:34:36,ИТ22.13,0,0.00,14.0
i22s0613,26,151392344,2022-09-13 19:46:35.077,00:19:10,ИТ22.12,17,94.44,NaN
i22s0615,15,151420319,2022-09-13 23:14:21.140,00:29:10,ИТ22.11,12,66.67,NaN
i22s0616,13,151424634,2022-09-13 23:54:31.811,00:59:36,ИТ22.12,13,72.22,NaN
i22s0617,40,151271145,2022-09-12 23:11:25.577,00:52:34,ИТ22.11,14,77.78,NaN
...,...,...,...,...,...,...,...,...
i22s0998,41,151267401,2022-09-12 22:33:25.769,00:28:36,ИТ22.11,8,44.44,NaN
i22s0999,78,151070398,2022-09-10 15:59:51.697,00:23:58,ИТ22.13,15,83.33,12.5
i22s1293,16,151418536,2022-09-13 23:00:00.877,00:11:38,ИТ22.11,12,66.67,NaN
